In [ ]:
#Importing the necessary libraries
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Importing the beautiful soup library
import bs4
import requests as rq

In [ ]:
#Installing the wikipedia and neuralcoref library
! pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Importing the wikipedia library to scrape text from wikipedia
import wikipedia as wk

In [ ]:
#Step-1: Automating Text extraction from the respective wikipedia pages - 
#Sub-step(i): Taking input of the wikipedia page name from user
#Sub-step(ii): Re-organizing the text into paragraphs without headers
#Sub-step(iii): Taking input of the word upto which the extract to be taken
#Sub-step(iv): Writing the extract into a text file

In [ ]:
#(i)
name=input("Enter the name of the wikipedia page:")
wiki=wk.page(name)
unfil_text=wiki.content

Enter the name of the wikipedia page:paracetamol


In [ ]:
#(ii)
#Importing Regular Expressions library
import re
text = re.sub(r'==.*?==+', '', unfil_text)
text = text.replace('\n', '')
print(text)

Paracetamol,  also known as acetaminophen, is a medication used to treat fever and mild to moderate pain. Common brand names include Tylenol and Panadol.At a standard dose, paracetamol only slightly decreases body temperature; it is inferior to ibuprofen in that respect, and the benefits of its use for fever are unclear. Paracetamol may relieve pain in acute mild migraine but only slightly in episodic tension headache. However, the aspirin/paracetamol/caffeine combination helps with both conditions where the pain is mild and is recommended as a first-line treatment for them. Paracetamol is effective for post-surgical pain, but it is inferior to ibuprofen. The paracetamol/ibuprofen combination provides further increase in potency and is superior to either drug alone. The pain relief paracetamol provides in osteoarthritis is small and clinically insignificant. The evidence in its favor for the use in low back pain, cancer pain, and neuropathic pain is insufficient.In the short term, comm

In [ ]:
#(iii)
extract_word=input("Enter word/digit/phrase upto which extract is to be taken:")
num=text.find(extract_word)
l=len(extract_word)
b=input('Enter name of file with extension:')
with open(b,'w') as fil:
  fil.write(text[:(num+l+1)])

Enter word/digit/phrase upto which extract is to be taken:prescriptions
Enter name of file with extension:new.txt


In [ ]:
final_text=text[:num+l+1]

In [ ]:
#Step-2: Coreference Resolution: Connecting all the pronouns to referred entities - 
#Sub-step(i): Utilising the allen nlp public models to relate all pronouns to the root word
#Sub-step(ii): Obtaining the spacy tokens for the words
#Sub-step(iii): Taking input of the word upto which the extract to be taken

In [ ]:
from allennlp.predictors.predictor import Predictor

model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)


prediction = predictor.predict(document=final_text)  # get prediction
print("indices-")
for cluster in prediction['clusters']:
    print(cluster)  # list the indices of spaCy tokens)

print('\n\n')

print('Coref resolved: ',predictor.coref_resolved(final_text))

In [ ]:
d=input('Enter name of file with extension:')
with open(d,'w') as f:
  f.write(predictor.coref_resolved(final_text))

In [ ]:
#Step-3: Entity Linking: Extracting all mentioned entities and connecting them to target knowledge base - 

In [ ]:
import urllib
from string import punctuation
import nltk

In [ ]:
import json

In [ ]:
def wikilink(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    results = list()
    for annotation in response["annotations"]:
        results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'],'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [ ]:
with open('newfile.txt','r') as file1:
  filtered_text=file1.read().rstrip()

In [ ]:
print(filtered_text)

Paracetamol,  also known as acetaminophen, is a medication used to treat fever and mild to moderate pain. Common brand names include Tylenol and Panadol.At a standard dose, paracetamol only slightly decreases body temperature; it is inferior to ibuprofen in that respect, and the benefits of its use for fever are unclear. Paracetamol may relieve pain in acute mild migraine but only slightly in episodic tension headache. However, the aspirin/paracetamol/caffeine combination helps with both conditions where the pain is mild and is recommended as a first-line treatment for them. Paracetamol is effective for post-surgical pain, but it is inferior to ibuprofen. The paracetamol/ibuprofen combination provides further increase in potency and is superior to either drug alone. The pain relief paracetamol provides in osteoarthritis is small and clinically insignificant. The evidence in its favor for the use in low back pain, cancer pain, and neuropathic pain is insufficient.In the short term, comm

In [ ]:
wikilink(filtered_text,threshold=1)


[{'title': 'Paracetamol',
  'wikiId': 'Q57055',
  'characters': [(0, 10),
   (28, 40),
   (173, 183),
   (323, 333),
   (444, 454),
   (582, 592),
   (668, 678),
   (793, 803),
   (1019, 1029),
   (1141, 1151),
   (1482, 1492),
   (1693, 1703),
   (1740, 1750),
   (1871, 1881),
   (2307, 2317),
   (2722, 2732),
   (2811, 2817)]},
 {'title': 'Paracetamol poisoning',
  'wikiId': 'Q2572879',
  'characters': [(2307, 2327)]},
 {'title': 'Medication',
  'wikiId': 'Q12140',
  'characters': [(48, 57),
   (765, 768),
   (1499, 1502),
   (2418, 2421),
   (2584, 2593),
   (2711, 2719),
   (2760, 2769),
   (2884, 2893)]},
 {'title': 'Prescription drug',
  'wikiId': 'Q1643563',
  'characters': [(2873, 2882), (2942, 2954)]},
 {'title': 'Used car',
  'wikiId': 'Q1137287',
  'characters': [(59, 62), (2579, 2582)]},
 {'title': 'English markers of habitual aspect',
  'wikiId': 'Q20991062',
  'characters': [(59, 65)]},
 {'title': 'Therapy',
  'wikiId': 'Q179661',
  'characters': [(67, 71), (551, 560), (5

In [ ]:
import itertools

In [ ]:
!git clone https://github.com/thunlp/OpenNRE.git

Cloning into 'OpenNRE'...
remote: Enumerating objects: 1518, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1518 (delta 82), reused 105 (delta 71), pack-reused 1377
Receiving objects: 100% (1518/1518), 266.83 MiB | 45.24 MiB/s, done.
Resolving deltas: 100% (903/903), done.


In [ ]:
!pip install opennre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement opennre (from versions: none)
ERROR: No matching distribution found for opennre


In [ ]:
!python setup.py install 

python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
import opennre as O

ModuleNotFoundError: ignored

In [ ]:
links = wikilink(filtered_text,threshold=1)
model = O.get_model('wiki80_cnn_softmax')
# Iterate over every permutation pair of entities
for permutation in itertools.permutations(links, 2):
  for source in permutation[0]['characters']:
        for target in permutation[1]['characters']:
            # Relationship extraction with OpenNRE
            data = model.infer(
                {'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})

AttributeError: ignored